In [6]:
from helpers import *
import xgboost as xgb
from icecream import ic

In [7]:
data = np.genfromtxt('data.txt')
data = data.reshape((data.shape[0] // 21, 21, data.shape[1]))

In [8]:
for i, hand in enumerate(data):
    data[i] = normalize_hand(hand)

In [9]:
hand_sample = data[0, :, :]
np.random.shuffle(data)
data = data.T
X, y = data[:-1].T, data[-1].T
split = int(.80 * X.shape[0])
X_train, X_test = X[:split, :], X[split:, :]
y_train, y_test = y[:split], y[split:]

In [10]:
for fingers_count in range(6):
    ic(y[y == fingers_count].shape[0])

# Balance data
y_shapes = []
temp_y = y[:, 0]
unbal_indices = []
for fingers_count in range(np.unique(temp_y).shape[0]):
    unbal_indices.append(np.where(temp_y == fingers_count)[0])
    shape_ = unbal_indices[fingers_count].shape[0]
    y_shapes.append(shape_)
ic(y_shapes)
bal_indices = []
min_shape = min(y_shapes)
for fingers_count in range(len(unbal_indices)):
    bal_indices.append(unbal_indices[fingers_count][0:min_shape])
bal_indices = np.array(bal_indices).flatten()
ic(bal_indices.shape)
y_new = y[bal_indices]
X_new = X[bal_indices]
ic(X_new.shape)
ic(y_new.shape)

ic| y[y == fingers_count].shape[0]: 25011
ic| y[y == fingers_count].shape[0]: 26523
ic| y[y == fingers_count].shape[0]: 26544
ic| y[y == fingers_count].shape[0]: 26502
ic| y[y == fingers_count].shape[0]: 26355
ic| y[y == fingers_count].shape[0]: 26481
ic| y_shapes: [1191, 1263, 1264, 1262, 1255, 1261]
ic| bal_indices.shape: (7146,)
ic| X_new.shape: (7146, 21, 3)
ic| y_new.shape: (7146, 21)


(7146, 21)

In [11]:
X = X_new
y = y_new

for fingers_count in range(6):
    ic(y[y == fingers_count].shape[0])

ic| y[y == fingers_count].shape[0]: 25011
ic| y[y == fingers_count].shape[0]: 25011
ic| y[y == fingers_count].shape[0]: 25011
ic| y[y == fingers_count].shape[0]: 25011
ic| y[y == fingers_count].shape[0]: 25011
ic| y[y == fingers_count].shape[0]: 25011


In [12]:
model = xgb.XGBClassifier()

In [13]:
# Flatten for preprocessing, training and testing
X_train = X_train.reshape(X_train.shape[0] * X_train.shape[1], X_train.shape[-1])
y_train = np.ndarray.flatten(y_train)

X_test = X_test.reshape(X_test.shape[0] * X_test.shape[1], X_test.shape[-1])
y_test = np.ndarray.flatten(y_test)

In [14]:
ic(X_train.shape)
ic(X_test.shape)

ic| X_train.shape: (125916, 3)
ic| X_test.shape: (31500, 3)


(31500, 3)

In [15]:
model.fit(X_train, y_train)

C:\Users\zackj\PycharmProjects\Hand Tracking\venv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:51:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [16]:
prediction = model.predict(X_test)
accuracy = np.average(y_test == prediction)
print('Accuracy: ', accuracy)

Accuracy:  0.8874920634920634


In [17]:
save_model(model, 'finalized_model.sav')